<a href="https://colab.research.google.com/github/Debapriya-Tula/Video-inbetweening/blob/master/Video_In_Betweening.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
! wget --no-check-certificate https://www.crcv.ucf.edu/data/UCF101/UCF101.rar

--2020-01-19 16:39:57--  https://www.crcv.ucf.edu/data/UCF101/UCF101.rar
Resolving www.crcv.ucf.edu (www.crcv.ucf.edu)... 132.170.214.127
Connecting to www.crcv.ucf.edu (www.crcv.ucf.edu)|132.170.214.127|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6932971618 (6.5G) [application/rar]
Saving to: ‘UCF101.rar’

UCF101.rar          100%[===================>]   6.46G  28.8MB/s    in 2m 26s  

2020-01-19 16:42:23 (45.4 MB/s) - ‘UCF101.rar’ saved [6932971618/6932971618]



In [0]:
! unrar e /content/UCF101.rar /content/UCF/

In [0]:
import numpy as np
import matplotlib.pyplot as plt
from keras import models, layers

%matplotlib inline
%tensorflow_version 1.x

Using TensorFlow backend.


In [0]:
x = layers.Input(shape=(2, 3))

tensorflow.python.framework.ops.Tensor

In [0]:
Conv_64_4_2 = layers.Conv2D(filters=64, kernel_size=(4, 4), strides=(2, 2))
Conv_64_3_1 = layers.Conv2D(filters=64, kernel_size=(3, 3), strides=(1, 1))
Conv_128_4_2 = layers.Conv2D(filters=128, kernel_size=(4, 4), strides=(2, 2))
Conv_128_3_1 = layers.Conv2D(filters=128, kernel_size=(3, 3), strides=(1, 1))
Conv_256_4_2 = layers.Conv2D(filters=256, kernel_size=(4, 4), strides=(2, 2))
Conv_256_3_1 = layers.Conv2D(filters=256, kernel_size=(3, 3), strides=(1, 1))
Conv_512_4_2 = layers.Conv2D(filters=512, kernel_size=(4, 4), strides=(2, 2))

pad_zeros_1 = lambda x: layers.ZeroPadding2D(padding=1)(x)
pad_zeros = lambda x: layers.ZeroPadding3D(padding=(0, 1, 1))(x)
vid_disc_layer = lambda filters: layers.Conv3D(filters=filters, kernel_size=4, strides=(1, 2, 2))

def shortcut(x, filters):
    x = layers.AveragePooling2D(pool_size=(2, 2))(x)
    x = layers.Conv2D(filters=filters, kernel_size=1, strides=1)(x)
    return x


In [0]:
# loss function
def vid_disc_loss(y_true, y_pred):
    return K.mean(-K.log(y_true) - K.log(1 - y_pred))

def img_disc_loss(y_true, y_pred):
    return K.mean(1/(T-2) * K.sum(-K.log(y_true) - K.log(1 - y_pred)))

def g_loss(vid_pred, img_pred):
    return K.mean(-K.log(vid_pred) - 1/(T-2) * K.sum(K.log(img_pred)))


def get_img_enc(shape):
    X_in = layers.Input(shape=shape)
    L1 = Conv_64_4_2(pad_zeros_1(X_in))
    L2 = Conv_64_3_1(pad_zeros_1(L1))
    L3 = Conv_128_4_2(pad_zeros_1(L2))
    L4 = Conv_128_3_1(pad_zeros_1(L3))
    L5 = Conv_256_4_2(pad_zeros_1(L4))
    L6 = Conv_256_3_1(pad_zeros_1(L5))
    L7 = Conv_64_3_1(pad_zeros_1(L6))

    return models.Model(inputs=X_in, outputs=L7)


def get_video_gen(shape, mode='rgb'):
    """
    mode: color mode of training
            possible values: rgb, gray
    """
    TConv_256_3_1 = layers.TransposedConv3D(filters=256, kernel_size=3, strides=1)
    X_in = layers.Input(shape=shape)
    L1 = TConv_256_3_1(pad_zeros_1(X_in))
    L2 = TConv_256_3_1(pad_zeros_1(L1))
    L3 = TransposedConv3D(filters=128, kernel_size=(3, 4, 4), strides=(1, 2, 2))(pad_zeros_1(L2))
    L4 = TransposedConv3D(filters=128, kernel_size=3, strides=1)(pad_zeros_1(L3))
    L5 = TransposedConv3D(filters=64, kernel_size=(3, 4, 4), strides=(1, 2, 2))(pad_zeros_1(L4))
    L6 = TransposedConv3D(filters=64, kernel_size=3, strides=1)(pad_zeros_1(L5))
    
    if mode == 'rgb':
        filters = 3
    elif mode == 'gray':
        filters = 1
    else:
        raise
    
    L7 = TransposedConv3D(filters=filters, kernel_size=(3, 4, 4), strides=(1, 2, 2))(pad_zeros_1(L6))

    model = models.Model(inputs=X_in, outputs=L7)
    model.compile(optimizer='adam', loss=g_loss)
    return model


def get_video_disc(shape):
    X_in = layers.Input(shape=shape)
    L1 = vid_disc_layer(64)(pad_zeros(X_in))
    L2 = vid_disc_layer(128)(pad_zeros(L1))
    L3 = vid_disc_layer(256)(pad_zeros(L2))
    L4 = vid_disc_layer(512)(pad_zeros(L3))
    L5 = layers.Flatten()(L4)
    L5 = layers.Dense(1, activation='sigmoid')(L5)

    model = models.Model(inputs=X_in, outputs=L5)
    model.compile(optimizer='adam', loss=vid_disc_loss)
    return model


def get_img_disc(shape):
    X_in = layers.Input(shape=shape)
    L1 = layers.Conv2D(filters=shape[-1], kernel_size=3, strides=1)(pad_zeros_1(X_in))
    L2 = Conv_64_4_2(pad_zeros_1(L1)) + shortcut(X_in, 64)
    L3 = Conv_64_3_1(pad_zeros_1(L2))
    L4 = Conv_128_4_2(pad_zeros_1(L3)) + shortcut(X_in, 128)
    L5 = Conv_128_3_1(pad_zeros_1(L4))
    L6 = Conv_256_4_2(pad_zeros_1(L5)) + shortcut(X_in, 256)
    L7 = Conv_256_3_1(pad_zeros_1(L6))
    L8 = Conv_512_4_2(pad_zeros_1(L7)) + shortcut(X_in, 512)
    L9 = layers.Flatten()(L8)
    L9 = layers.Dense(1, activation='sigmoid')(L9)

    model = models.Model(inputs=X_in, outputs=L9)
    model.compile(optimizer='adam', loss=img_disc_loss)
    return model


def LRG_block(noise, start_enc, end_enc, z_in, T, block_no):
    C = 64
    H, W = map(lambda x: x.value, start.shape[:-1])

    if block_no in [1, 9, 17]:
        start_enc = layers.UpSampling2D(size=2)(start_enc)
        end_enc = layers.UpSampling2D(size=2)(end_enc)

    u = layers.Dense(T * C)(noise) # because C = 64
    u = layers.Reshape((T, C))(u)

    # gating operations
    gs = layers.Conv2D(filters=C, kernel_size=3, activation='sigmoid')(u)
    ge = layers.Conv2D(filters=C, kernel_size=3, activation='sigmoid')(u)

    # broadcasting tensors
    def broadcast_g(tensor):
        tensor = K.expand_dims(tensor, axis=1)
        tensor = K.repeat_elements(tensor, W, axis=1)
        tensor = K.expand_dims(tensor, axis=1)
        tensor = K.repeat_elements(tensor, H, axis=1)
        return tensor
    
    def broadcast_e(tensor):
        tensor = K.expand_dims(tensor, axis=0)
        tensor = K.repeat_elements(tensor, T, axis=0)
        return tensor
    
    gs = broadcast_g(gs)
    ge = broadcast_g(ge)

    E_s = broadcast_e(start_enc)
    E_e = broadcast_e(end_enc)

    # secondary noise
    n = layers.Conv2D(filters=C, kernel_size=3)(u)
    n = broadcast_g(n)

    # part 1 of the output
    z_in_2 = K.Multiply([gs, E_s]) + K.Multiply([ge, E_e]) + (K.maximum(0, 1 - gs - ge) * z_in) + n

    # part 2 of the output
    part1 = layers.LeakyReLU(0.2)(layers.Conv3D(filters=C, kernel_size=3)(z_in_2))
    part2 = layers.Conv3D(filters=C, kernel_size=3)(part1) + z_in_2
    z_out = layers.LeakyReLU(0.2)(part2)
    
    return z_out

In [0]:
# creating the final model
def get_final_model(shape, img_enc_model, vid_gen_model, vid_dsc_model, img_disc_model):
    start_frame = layers.Input(shape=shape)
    end_frame = layers.Input(shape=shape)

    # get encodings of start and end frames
    x_s = img_enc_model(start_frame)
    x_e = img_enc_model(end_frame)

    # coarse to fine encoding generation
    T = 4
    z_prev = K.concatenate([x_s, x_e], axis=-1)
    for l in range(1, 25):
        if l % 8 == 0:
            T *= 2
        
        noise_input = np.random.normal(size=(128, 1))
        noise_input = K.cast(noise_input, dtype="float32")

        z_new = LRG_block(noise_input, x_s, x_e, z_prev, T, l)
    
    # generate video sequence
    vid_shape = (T,) + shape
    vid_seq = vid_gen_model(z_new)

    # feed sequence to discriminator
    vid_disc_model.trainable = False
    vid_predictions = vid_disc_model(vid_seq)

    # build the model
    model = models.Model(inputs=[start_frame, end_frame], outputs=vid_predictions)
    return model

In [0]:
vid_disc = get_video_disc()
def training(epochs=100, batch_size=128):
    # loading the data

    # Creating the GAN

    for epoch in range(epochs):
        print(f"Epoch: {epoch}")

        # generate random noise


In [0]:
# import os
# from google.colab import files
# files.download('/content/UCF/'+os.listdir('UCF')[0])

In [0]:
# from torch import cuda, nn, optim
# class ImageEncoder(nn.Module):
#     def __init__(self, input_shape, *args, **kwargs):
#         super(ImageEncoder, self).__init__(*args, **kwargs)

#         self.Conv_channels_64_4_2_1 = nn.Conv2d(in_channels=input_shape[0], out_channels=64, kernel_size=(4, 4), stride=2, padding=1)
#         self.Conv_64_64_3_1_1 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=(3, 3), stride=1, padding=1)
#         self.Conv_64_128